In [1]:
#here we will write code to apply svm for music genre classification

import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load GTZAN Feature CSV
df = pd.read_csv("features_30_sec.csv")

#reading the data
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [2]:
#to know the name of the columns
df.columns

Index(['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var',
  

In [4]:
# Select Features
features = [
    # Spectral Features
    "chroma_stft_mean", "chroma_stft_var",
    "spectral_centroid_mean", "spectral_centroid_var",
    "spectral_bandwidth_mean", "spectral_bandwidth_var",
    "rolloff_mean", "rolloff_var",
    "zero_crossing_rate_mean", "zero_crossing_rate_var",

    # Harmonic & Perceptual Features
    "harmony_mean", "harmony_var",
    "perceptr_mean", "perceptr_var",

    # Rhythmic Feature
    "tempo"
]

# Add MFCC Mean and Variance (Both Improve SVM Performance)
mfcc_mean_features = [f"mfcc{i}_mean" for i in range(1, 21)]
mfcc_var_features = [f"mfcc{i}_var" for i in range(1, 21)]
features.extend(mfcc_mean_features)
features.extend(mfcc_var_features)

# Extract Features and Labels
X = df[features]
y = df['label']

# Encode Genre Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Train-Test Split (80-20 Split)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Normalize Features for SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM Model (Optimized Hyperparameters)
svm_model = SVC(kernel='rbf', C=15, gamma='scale', probability=True)
svm_model.fit(X_train_scaled, y_train)

# Predict Using SVM
svm_preds = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_preds)

# Print Accuracy in percentage
print(f"\nAccuracy for SVM: {svm_accuracy * 100:.2f}%")

# Print Classification Report
print("\nClassification Report for SVM:")
print(classification_report(y_test, svm_preds, target_names=label_encoder.classes_))


Accuracy for SVM: 77.00%

Classification Report for SVM:
              precision    recall  f1-score   support

       blues       0.77      0.85      0.81        20
   classical       0.81      0.85      0.83        20
     country       0.85      0.85      0.85        20
       disco       0.71      0.60      0.65        20
      hiphop       0.81      0.85      0.83        20
        jazz       0.70      0.80      0.74        20
       metal       0.94      0.80      0.86        20
         pop       0.74      0.70      0.72        20
      reggae       0.68      0.75      0.71        20
        rock       0.72      0.65      0.68        20

    accuracy                           0.77       200
   macro avg       0.77      0.77      0.77       200
weighted avg       0.77      0.77      0.77       200

